In [1]:
import duckdb
from pathlib import Path

In [ ]:
conn = duckdb.connect()

conn.sql("""
INSTALL sqlite;
LOAD sqlite;
CALL sqlite_attach('../data/sqlite-sakila.db');
""")

conn.sql("SELECT * FROM film LIMIT 5").df()

,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,<NA>,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2021-03-06 15:52:00
1,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,2006,1,<NA>,3,4.99,48,12.99,G,"Trailers,Deleted Scenes",2021-03-06 15:52:00
2,3,ADAPTATION HOLES,A Astounding Reflection of a Lumberjack And a ...,2006,1,<NA>,7,2.99,50,18.99,NC-17,"Trailers,Deleted Scenes",2021-03-06 15:52:00
3,4,AFFAIR PREJUDICE,A Fanciful Documentary of a Frisbee And a Lumb...,2006,1,<NA>,5,2.99,117,26.99,G,"Commentaries,Behind the Scenes",2021-03-06 15:52:00
4,5,AFRICAN EGG,A Fast-Paced Documentary of a Pastry Chef And ...,2006,1,<NA>,6,2.99,130,22.99,G,Deleted Scenes,2021-03-06 15:52:00


### Movies longer than 3 hours

In [17]:
q_a = conn.sql("""
    SELECT title, length
    FROM film
    WHERE length > 180
    ORDER BY length DESC;
""").df()

q_a.head()

,title,length
0,WORST BANGER,185
1,CHICAGO NORTH,185
2,CONTROL ANTHEM,185
3,DARN FORRESTER,185
4,SWEET BROTHERHOOD,185


### Movies with the word "love" in its title

In [18]:
q_b = conn.sql("""
    SELECT
        title,
        rating,
        length,
        description
    FROM film
    WHERE LOWER(title) LIKE '%love%'
    ORDER BY title;
""").df()

q_b

,title,rating,length,description
0,GRAFFITI LOVE,PG,117,A Unbelieveable Epistle of a Sumo Wrestler And...
1,IDAHO LOVE,PG-13,172,A Fast-Paced Drama of a Student And a Crocodil...
2,IDENTITY LOVER,PG-13,119,A Boring Tale of a Composer And a Mad Cow who ...
3,INDIAN LOVE,NC-17,135,A Insightful Saga of a Mad Scientist And a Mad...
4,LAWRENCE LOVE,NC-17,175,A Fanciful Yarn of a Database Administrator An...
5,LOVE SUICIDES,R,181,A Brilliant Panorama of a Hunter And a Explore...
6,LOVELY JINGLE,PG,65,A Fanciful Yarn of a Crocodile And a Forensic ...
7,LOVER TRUMAN,G,75,A Emotional Yarn of a Robot And a Boy who must...
8,LOVERBOY ATTACKS,PG-13,162,A Boring Story of a Car And a Butler who must ...
9,STRANGELOVE DESIRE,NC-17,103,A Awe-Inspiring Panorama of a Lumberjack And a...


### descriptive statistics

In [19]:
q_c = conn.sql("""
    SELECT
        MIN(length)    AS shortest,
        AVG(length)    AS average,
        MEDIAN(length) AS median,
        MAX(length)    AS longest
    FROM film;
""").df()

q_c

,shortest,average,median,longest
0,46,115.272,114.0,185


### Rental rate

In [20]:
q_d = conn.sql("""
    SELECT
        title,
        rental_rate,
        rental_duration,
        rental_rate / rental_duration AS rate_per_day
    FROM film
    ORDER BY rate_per_day DESC
    LIMIT 10;
""").df()

q_d

,title,rental_rate,rental_duration,rate_per_day
0,AMERICAN CIRCUS,4.99,3,1.663333
1,BACKLASH UNDEFEATED,4.99,3,1.663333
2,BILKO ANONYMOUS,4.99,3,1.663333
3,BEAST HUNCHBACK,4.99,3,1.663333
4,CARIBBEAN LIBERTY,4.99,3,1.663333
5,AUTUMN CROW,4.99,3,1.663333
6,CASPER DRAGONFLY,4.99,3,1.663333
7,ACE GOLDFINGER,4.99,3,1.663333
8,BEHAVIOR RUNAWAY,4.99,3,1.663333
9,CASUALTIES ENCINO,4.99,3,1.663333


### Actors have played in the most movies

In [21]:
q_e = conn.sql("""
    SELECT
        a.actor_id,
        a.first_name,
        a.last_name,
        COUNT(*) AS num_movies
    FROM actor a
    JOIN film_actor fa
        ON a.actor_id = fa.actor_id
    GROUP BY a.actor_id, a.first_name, a.last_name
    ORDER BY num_movies DESC
    LIMIT 10;
""").df()

q_e

,actor_id,first_name,last_name,num_movies
0,107.0,GINA,DEGENERES,42
1,102.0,WALTER,TORN,41
2,198.0,MARY,KEITEL,40
3,181.0,MATTHEW,CARREY,39
4,23.0,SANDRA,KILMER,37
5,81.0,SCARLETT,DAMON,36
6,144.0,ANGELA,WITHERSPOON,35
7,60.0,HENRY,BERRY,35
8,13.0,UMA,WOOD,35
9,106.0,GROUCHO,DUNST,35


### Any actor named Leo

In [ ]:
q_f = conn.sql("""
    SELECT
        actor_id,
        first_name,
        last_name
        description
    FROM actor
    WHERE LOWER(first_name) = 'leo'
    ORDER BY last_name;
""").df()

q_f

,actor_id,first_name,description


### Most common first name

In [33]:
q_g = conn.sql("""
    SELECT
        first_name,
        COUNT(*) AS count
    FROM actor
    GROUP BY first_name
    ORDER BY count DESC
    LIMIT 10;
""").df()

q_g
        

,first_name,count
0,PENELOPE,4
1,KENNETH,4
2,JULIA,4
3,NICK,3
4,RUSSELL,3
5,JAYNE,3
6,GROUCHO,3
7,CUBA,3
8,DAN,3
9,CHRISTIAN,3


### Best rating 

In [ ]:
films = conn.sql("""
    SELECT
        title,
        rating,
        length,
        rental_rate
    FROM film
""").df()

films.head()

,title,rating,length,rental_rate
0,ACADEMY DINOSAUR,PG,86,0.99
1,ACE GOLDFINGER,G,48,4.99
2,ADAPTATION HOLES,NC-17,50,2.99
3,AFFAIR PREJUDICE,G,117,2.99
4,AFRICAN EGG,G,130,2.99


: 